# Create MEGA SET
This is simply a horizontal concatenation of numerical, categorical, and time stamp data.  It will be insanely wide.  I suspect this will be so wide that it is impractical to use.
### Questions I want to answer:
Will permutation importances help me identify my most performant features?  
Can a model with over 2k columns be trained in any meaningful way?  
Will the gods strike me down for my hubris?  

# Evaluation
I will judge on precision and mathews coefficient.  I chose precision based off of a confusion matrix (see main notebook).  Mathews coefficient was the original competition score, and I'd like to see where I stack up.


## MEGA SET

In [1]:
#lets get started on megaset.  My methodology for this is going to read in a 
#chunk of each data
import pandas as pd

def get_iters():

    folder = 'bosch-production-line-performance/'

    #iterables for each data set
    num_iter = pd.read_csv(folder + 'train_numeric.csv', iterator = True, 
                           chunksize = 1000)

    date_iter = pd.read_csv(folder + 'train_date.csv', iterator = True, 
                            chunksize = 1000)
    
    #during exploration I discovered that the categorical data set is all 
    #strings, so reading it in as such.
    cat_iter = pd.read_csv(folder + 'train_categorical.csv', dtype = str, 
                           iterator = True, chunksize = 1000)
    
    return(num_iter, date_iter, cat_iter)


In [2]:
num_iter, date_iter, cat_iter = get_iters()

In [3]:
cat = cat_iter.get_chunk()

num = num_iter.get_chunk()

date = date_iter.get_chunk()

In [4]:
cat.shape, num.shape, date.shape

((1000, 2141), (1000, 970), (1000, 1157))

In [5]:
cat.head()

,Id,L0_S1_F25,L0_S1_F27,L0_S1_F29,L0_S1_F31,L0_S2_F33,L0_S2_F35,L0_S2_F37,L0_S2_F39,L0_S2_F41,...,L3_S49_F4225,L3_S49_F4227,L3_S49_F4229,L3_S49_F4230,L3_S49_F4232,L3_S49_F4234,L3_S49_F4235,L3_S49_F4237,L3_S49_F4239,L3_S49_F4240
0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
num.head()

,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,...,L3_S50_F4245,L3_S50_F4247,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262,Response
0,4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,9,-0.036,-0.064,0.294,0.330,0.074,0.161,0.022,0.128,-0.026,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,11,-0.055,-0.086,0.294,0.330,0.118,0.025,0.030,0.168,-0.169,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [7]:
date.head()

,Id,L0_S0_D1,L0_S0_D3,L0_S0_D5,L0_S0_D7,L0_S0_D9,L0_S0_D11,L0_S0_D13,L0_S0_D15,L0_S0_D17,...,L3_S50_D4246,L3_S50_D4248,L3_S50_D4250,L3_S50_D4252,L3_S50_D4254,L3_S51_D4255,L3_S51_D4257,L3_S51_D4259,L3_S51_D4261,L3_S51_D4263
0,4,82.24,82.24,82.24,82.24,82.24,82.24,82.24,82.24,82.24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,1618.70,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,1149.20,1149.20,1149.20,1149.20,1149.20,1149.20,1149.20,1149.20,1149.20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,602.64,602.64,602.64,602.64,602.64,602.64,602.64,602.64,602.64,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Verifying each dataset is in the same order
Okay, I know for sure each set as an ID column that I can index with, so lets go ahead and just verify that they actually line up.

In [8]:
folder = 'bosch-production-line-performance/'

catid = pd.read_csv(folder + 'train_categorical.csv', usecols = ['Id'], 
                    squeeze = True)
numid = pd.read_csv(folder + 'train_numeric.csv', usecols = ['Id'], 
                    squeeze = True)
dateid = pd.read_csv(folder + 'train_date.csv', usecols = ['Id'], 
                     squeeze = True)
catid.shape, numid.shape, dateid.shape

((1183747,), (1183747,), (1183747,))

In [9]:
#looking good so far
if catid.equals(numid) and numid.equals(dateid):
    print('Each data set is in the same order.')

Each data set is in the same order.


In [10]:
#resetting all variables with ID in them.  Need to clear up memory for megaset
%reset_selective -f id

#### Does each set have my target column?
According to my data's [documentation](https://www.kaggle.com/c/bosch-production-line-performance/data) we are trying to predict for "Response" but I don't know if each dataset has that feature so lets find out.

In [11]:
try:
    print(num['Response'])
except:
    print('No response column in the numeric dataset')
try:
    print(cat['Response'])
except:
    print('No reponse column in the categorical dataset')
try:
    print(date['Response'])
except:
    print('No response column in the date dataset')

0      0
1      0
2      0
3      0
4      0
      ..
995    0
996    0
997    0
998    0
999    0
Name: Response, Length: 1000, dtype: int64
No reponse column in the categorical dataset
No response column in the date dataset


### Build the mega set

In [12]:
#easy enough, lets just concat everything
mega = pd.merge(num, date, how = 'outer', on = 'Id')
mega.shape
mega.head()

,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,...,L3_S50_D4246,L3_S50_D4248,L3_S50_D4250,L3_S50_D4252,L3_S50_D4254,L3_S51_D4255,L3_S51_D4257,L3_S51_D4259,L3_S51_D4261,L3_S51_D4263
0,4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,-0.036,-0.064,0.294,0.330,0.074,0.161,0.022,0.128,-0.026,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,-0.055,-0.086,0.294,0.330,0.118,0.025,0.030,0.168,-0.169,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
#shaped as expected
mega['Id'].dtype

dtype('int64')

In [14]:
cat['Id'].dtype

dtype('O')

In [15]:
#okay not going to merge with different data types for identifiers.  Lets go 
#ahead and cast the categorical column to an int
cat['Id'] = cat['Id'].apply(int)

In [16]:
cat['Id'].dtype

dtype('int64')

In [17]:
#okay I'm expecting this to be 4268 - 2 'Id' columns removed
mega = pd.merge(mega, cat, how = 'outer', on = 'Id')
mega.shape
mega.head()

,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,...,L3_S49_F4225,L3_S49_F4227,L3_S49_F4229,L3_S49_F4230,L3_S49_F4232,L3_S49_F4234,L3_S49_F4235,L3_S49_F4237,L3_S49_F4239,L3_S49_F4240
0,4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,-0.036,-0.064,0.294,0.330,0.074,0.161,0.022,0.128,-0.026,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,-0.055,-0.086,0.294,0.330,0.118,0.025,0.030,0.168,-0.169,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
#I kinda have to keep tuning this as it kills my kernel, so the final numbers 
#are the max that my machine can handle.

mega_chunks = [mega]

for i in range(10):
    num = num_iter.get_chunk()
    date = date_iter.get_chunk()
    cat = cat_iter.get_chunk()
    
    cat['Id'] = cat['Id'].apply(int)
    a = pd.merge(num, date, how = 'outer', on = 'Id')
    mega_chunks.append(pd.merge(a, cat, how = 'outer', on = 'Id'))
    
mega = pd.concat(mega_chunks, ignore_index = True)

In [19]:
print(mega.shape)
mega.head()

(11000, 4266)


,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,...,L3_S49_F4225,L3_S49_F4227,L3_S49_F4229,L3_S49_F4230,L3_S49_F4232,L3_S49_F4234,L3_S49_F4235,L3_S49_F4237,L3_S49_F4239,L3_S49_F4240
0,4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9,-0.036,-0.064,0.294,0.330,0.074,0.161,0.022,0.128,-0.026,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11,-0.055,-0.086,0.294,0.330,0.118,0.025,0.030,0.168,-0.169,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
#Lets save this work, incase I want to revisit it later. A little time consuming
#but good for saving future time :D
#commented out, because I already did it :D
#mega.to_csv('wrangled-sets/MEGA_SET.csv', index = False)

In [23]:
#lets give it a quick prep for evaluation
from sklearn.model_selection import train_test_split
target = 'Response'
X = mega.drop(columns = ['Id', target])
y = mega[target]

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size = .33,
                                                    random_state = 42, 
                                                    stratify = mega[target])
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((7370, 4264), (3630, 4264), (7370,), (3630,))

## Evaluation Method
I will be using the same method to evaluate all three.
#### Baseline
I'll be setting my baseline at a "guess"
#### Hypothesis
Hypothesis is fundamentally the same for all things I'm testing: (MEGA SET, meta-features, stacking) will out perform my baseline in both precision and MattyCo.
#### Test
1. Simple linear classification model and evaluate Precision and MCC  
2. Simple tree based model and evaluate Precision and Mathews Coefficient
3. Validate with CV and light parameter tuning

#### Accept/Reject Hypothesis
I'll ultimately use the most performant method of wrangling my data for my final and more thorough analysis

### Baseline

In [24]:
#lets set up a dummy
from sklearn.dummy import DummyClassifier
from sklearn.metrics import precision_score, matthews_corrcoef

base_model = DummyClassifier(strategy = 'uniform', random_state = 42)

base_model.fit(X, y)

y_pred_base = base_model.predict(X)

def metrics(y_true, y_pred):
    #We are setting zero division to 0, because a dataset this imbalance
    #is very likely generate warnings otherwise.
    print('Our Precision is: ', precision_score(y_true, y_pred, 
                                               zero_division = 0))
    print('Our MCC is: ', matthews_corrcoef(y_true, y_pred))
    
metrics(y, y_pred_base)

Our baseline Precision is:  0.005456529647144416
Our baseline MCC is:  -0.0035865157676011174


### Hypothesis
MEGA SET method of wrangling will out perform my baseline in both precision 
and mcc when run through a standard test of two models and cross validation

### Tests - Linear

In [25]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier, XGBRFClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import category_encoders as ce

In [26]:
#going to do a quick test between a couple pipelines

log_pipe = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(),
    StandardScaler(),
    LogisticRegression(n_jobs = -1, random_state = 42)   
)

#we are going to use early stopping, which is why we have such a high estimator
XGBC_pipe = make_pipeline(
    ce.OrdinalEncoder(),
    StandardScaler(),
    XGBClassifier(booster = 'gblinear', n_estimators = 1000, n_jobs = -1, 
                  random_state = 42)
)

In [27]:
from scipy.stats import uniform, randint

#very light tuning, based off of a few research articles on what parameters are
#best for each model

log_param = {
    #'simpleimputer__strategy': ['mean', 'median'],
    'logisticregression__C': [.001, .01, .1, 1, 10, 100, 1000]
}

XGBC_param = {
    'xgbclassifier__max_depth': randint(1, 50),
    'xgbclassifier__min_child_weight': randint(1, 50),
    'xgbclassifier__gamma': uniform(0, 1)
}

In [28]:
#lets start with logistic first
log_search = RandomizedSearchCV(
    log_pipe,
    param_distributions = log_param,
    n_iter = 7,
    cv = 5,
    scoring = 'precision',
    verbose = 10,
    return_train_score = True,
    n_jobs = -1
)

log_search.fit(X_train, y_train);

Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed: 38.1min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed: 39.0min
/home/felix/anaconda3/envs/anaconda_env/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:703: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  16 out of  35 | elapsed: 77.4min remaining: 91.9min
[Parallel(n_jobs=-1)]: Done  20 out of  35 | elapsed: 78.1min remaining: 58.5min
[Parallel(n_jobs=-1)]: Done  24 out of  35 | elapsed: 78.4min remaining: 35.9min
[Parallel(n_jobs=-1)]: Done  28 out of  35 | elapsed: 113.0min remaining: 28.2min
[Parallel(n_jobs=-1)]: Done  32 out of  35 | elapsed: 113.2min remaining: 10.6min
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed: 113.5min finished


In [29]:
best_log = log_search.best_estimator_

In [30]:
y_pred_log = best_log.predict(X_test)

metrics(y_test, y_pred_log)

Our baseline Precision is:  0.0
Our baseline MCC is:  -0.0021938304286995766


In [ ]:
# #hahahaha, oh thats what I expected. I really had to cut the rows just to get
# #this to fit into memory.  Its just not enough obeservations for how imbalanced
# #this dataset is.

### Reject Hypothesis
This data set is simply too large (wide) to work with.  I have put 6 hours into making it work and by the time I did, there were too few observations to make a model that could beat baseline. I'm calling this experiment early. I wanted to know if the gods would strike me down for my hubris. Well they done shot me in my HEEL! MEGA SET you are too t h i c c for for your own good.